In [1]:
#from https://www.tutorialspoint.com/python_data_access/python_postgresql_create_database.htm
#establishing the connection
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://root"+':'+pw+'@'+host_var+":5432"+'/postgres'

In [3]:
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


# Create Tables in Database

### CSV from USGS

In [8]:
import pandas as pd
#load csv

df = pd.read_csv('Data_Extraction_Transform/all_month(3).csv', index_col=0)
df.head(5)

,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
time,,,,,,,,,,,,,,,,,,,,,
2021-08-19T02:53:29.951Z,38.541200,-119.463900,3.60,1.30,ml,9.0,58.29,0.04200,0.1139,nn,...,2021-08-19T02:56:56.695Z,"3 km NNE of Walker, California",earthquake,NaN,0.90,0.130,4.0,automatic,nn,nn
2021-08-19T02:49:35.880Z,35.896667,-117.729167,4.48,0.72,ml,9.0,72.00,0.08187,0.0900,ci,...,2021-08-19T02:53:05.273Z,"17km ESE of Little Lake, CA",earthquake,0.26,0.87,0.121,8.0,automatic,ci,ci
2021-08-19T02:49:14.046Z,59.865100,-153.176400,102.60,1.60,ml,NaN,NaN,NaN,0.9900,ak,...,2021-08-19T03:02:14.791Z,"52 km E of Pedro Bay, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak
2021-08-19T02:46:16.400Z,38.784668,-122.751000,0.33,1.03,md,13.0,84.00,0.01125,0.0100,nc,...,2021-08-19T02:47:54.191Z,"1km NNE of The Geysers, CA",earthquake,0.24,0.45,0.160,4.0,automatic,nc,nc
2021-08-19T02:42:46.880Z,56.657400,-157.265100,0.50,2.90,ml,NaN,NaN,NaN,0.6300,ak,...,2021-08-19T02:50:26.938Z,"80 km ENE of Chignik, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak


In [9]:
df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

### Clean CSV for models

In [11]:

df_modelRF = pd.read_csv('Data_Extraction_Transform/DroppedTestwithRMSandNST.csv', index_col=0)
df_modelRF.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,rms,net,id,updated,place,type,magNst,locationSource
0,2021-08-19T02:53:29.951Z,38.541200,-119.463900,3.60,1.30,ml,9,58.29,0.1139,nn,nn00818986,2021-08-19T02:56:56.695Z,"3 km NNE of Walker, California",earthquake,4,nn
1,2021-08-19T02:49:35.880Z,35.896667,-117.729167,4.48,0.72,ml,9,72.00,0.0900,ci,ci40014776,2021-08-19T02:53:05.273Z,"17km ESE of Little Lake, CA",earthquake,8,ci
3,2021-08-19T02:46:16.400Z,38.784668,-122.751000,0.33,1.03,md,13,84.00,0.0100,nc,nc73611791,2021-08-19T02:47:54.191Z,"1km NNE of The Geysers, CA",earthquake,4,nc
5,2021-08-19T02:39:51.510Z,34.032167,-116.749500,9.15,2.29,ml,65,44.00,0.1800,ci,ci40014768,2021-08-19T02:50:20.370Z,"13km NNE of Cabazon, CA",earthquake,27,ci
6,2021-08-19T02:37:55.960Z,35.456667,-118.314500,3.21,1.45,ml,20,57.00,0.1800,ci,ci40014760,2021-08-19T02:41:43.609Z,"22km SE of Bodfish, CA",earthquake,26,ci


In [13]:
df_modelRF.to_sql('earthquake_rfmodel_data', engine, if_exists='replace', index = False)

In [14]:

df_modelKNN = pd.read_csv('Data_Extraction_Transform/csv_misc/dropped_cols_month_earthquake.csv', index_col=0)
df_modelKNN.head(5)

,id,tsunami,felt,time,latitude,longitude,depth,mag,magType,gap,net,updated,place,type,type_encoder,locationSource
0,nn00818986,0,NaN,2021-08-19T02:53:29.951Z,38.541200,-119.463900,3.60,1.30,ml,58.29,nn,2021-08-19T02:56:56.695Z,"3 km NNE of Walker, California",earthquake,1,nn
1,ci40014776,0,NaN,2021-08-19T02:49:35.880Z,35.896667,-117.729167,4.48,0.72,ml,72.00,ci,2021-08-19T02:53:05.273Z,"17km ESE of Little Lake, CA",earthquake,1,ci
2,ak021am2mkg5,0,NaN,2021-08-19T02:49:14.046Z,59.865100,-153.176400,102.60,1.60,ml,NaN,ak,2021-08-19T03:02:14.791Z,"52 km E of Pedro Bay, Alaska",earthquake,1,ak
3,nc73611791,0,NaN,2021-08-19T02:46:16.400Z,38.784668,-122.751000,0.33,1.03,md,84.00,nc,2021-08-19T02:47:54.191Z,"1km NNE of The Geysers, CA",earthquake,1,nc
4,ak021am2l4yc,0,NaN,2021-08-19T02:42:46.880Z,56.657400,-157.265100,0.50,2.90,ml,NaN,ak,2021-08-19T02:50:26.938Z,"80 km ENE of Chignik, Alaska",earthquake,1,ak


In [15]:
df_modelKNN.to_sql('earthquake_knnmodel_data', engine, if_exists='replace', index = False)

In [16]:

df_model_vizAP = pd.read_csv('Data_Extraction_Transform/csv_misc/locationtype_month_earthquakes.csv', index_col=0)
df_model_vizAP.head(5)

,id,tsunami,felt,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,place,type,magNst,Unix Time,grouped_type
0,nn00818986,0,NaN,2021-08-19 02:53:29.951000+00:00,38.541200,-119.463900,3.60,1.30,ml,9.0,58.29,0.04200,0.1139,nn,"3 km NNE of Walker, California",earthquake,4.0,1.629342e+09,1.0
1,ci40014776,0,NaN,2021-08-19 02:49:35.880000+00:00,35.896667,-117.729167,4.48,0.72,ml,9.0,72.00,0.08187,0.0900,ci,"17km ESE of Little Lake, CA",earthquake,8.0,1.629341e+09,1.0
2,ak021am2mkg5,0,NaN,2021-08-19 02:49:14.046000+00:00,59.865100,-153.176400,102.60,1.60,ml,NaN,NaN,NaN,0.9900,ak,"52 km E of Pedro Bay, Alaska",earthquake,NaN,1.629341e+09,1.0
3,nc73611791,0,NaN,2021-08-19 02:46:16.400000+00:00,38.784668,-122.751000,0.33,1.03,md,13.0,84.00,0.01125,0.0100,nc,"1km NNE of The Geysers, CA",earthquake,4.0,1.629341e+09,1.0
4,ak021am2l4yc,0,NaN,2021-08-19 02:42:46.880000+00:00,56.657400,-157.265100,0.50,2.90,ml,NaN,NaN,NaN,0.6300,ak,"80 km ENE of Chignik, Alaska",earthquake,NaN,1.629341e+09,1.0


In [18]:
df_model_vizAP.to_sql('earthquake_vizap_data', engine, if_exists='replace', index = False)

In [19]:

df_mag_depth_pred = pd.read_csv('ML_Predictions/predictions_full_data.csv', index_col=0)
df_mag_depth_pred.head(5)

,actual_depth,predicted_depth,actual_mag,predicted_mag,time,latitude,longitude,depth,mag,magType,...,gap,rms,net,id,updated,place,type,magNst,locationSource,unixTime
6497,-0.440000,-0.178000,1.74,1.745417,2021-07-22T11:01:11.570Z,19.351500,-155.136833,-0.440000,1.74,md,...,110.0,0.14,hv,hv72597887,2021-07-22T22:31:09.890Z,"12 km S of Fern Forest, Hawaii",earthquake,12,hv,1.626952e+09
8186,4.280000,5.078857,0.36,0.341458,2021-07-23T14:36:10.620Z,35.884667,-117.674500,4.280000,0.36,ml,...,88.0,0.12,ci,ci39750831,2021-07-23T18:23:16.496Z,"22km ESE of Little Lake, CA",earthquake,4,ci,1.627051e+09
6004,5.390000,5.013714,1.25,1.169375,2021-07-23T13:20:54.710Z,38.502500,-119.523167,5.390000,1.25,md,...,77.0,0.05,nc,nc73598156,2021-08-03T20:11:17.553Z,"32km SE of Markleeville, CA",earthquake,7,nc,1.627046e+09
9222,9.267944,9.233422,2.30,2.158333,2021-07-21T12:31:01.198Z,31.606910,-103.987350,9.267944,2.30,ml,...,112.0,0.20,tx,tx2021oeev,2021-07-21T16:32:42.527Z,"37 km NNW of Toyah, Texas",earthquake,7,tx,1.626871e+09
17684,8.908032,8.870572,1.90,1.925000,2021-07-20T05:40:02.338Z,31.606007,-103.982505,8.908032,1.90,ml,...,114.0,0.20,tx,tx2021obvv,2021-07-20T18:40:56.768Z,"36 km NNW of Toyah, Texas",earthquake,9,tx,1.626760e+09


In [20]:
df_mag_depth_pred.to_sql('earthquake_magdepth_predicted_data', engine, if_exists='replace', index = False)

In [21]:

df_types_pred = pd.read_csv('ML_Predictions/predicted_types.csv', index_col=0)
df_types_pred.head(5)

,Predicted Type,Actual Type
8870,1,1
5046,1,1
12788,1,1
3499,1,1
9965,1,1


In [22]:
df_types_pred.to_sql('earthquake_types_predicted_data', engine, if_exists='replace', index = False)

### Tables need modifying in PG Admin 4 to add primary key via "Alter Table"

# Read Tables from SQL

In [23]:
table_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
table_df

,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,unique_id
0,38.541200,-119.463900,3.60,1.30,ml,9.0,58.29,0.04200,0.1139,nn,...,"3 km NNE of Walker, California",earthquake,NaN,0.90,0.130000,4.0,automatic,nn,nn,1
1,35.896667,-117.729167,4.48,0.72,ml,9.0,72.00,0.08187,0.0900,ci,...,"17km ESE of Little Lake, CA",earthquake,0.26,0.87,0.121000,8.0,automatic,ci,ci,2
2,59.865100,-153.176400,102.60,1.60,ml,NaN,NaN,NaN,0.9900,ak,...,"52 km E of Pedro Bay, Alaska",earthquake,NaN,0.50,NaN,NaN,automatic,ak,ak,3
3,38.784668,-122.751000,0.33,1.03,md,13.0,84.00,0.01125,0.0100,nc,...,"1km NNE of The Geysers, CA",earthquake,0.24,0.45,0.160000,4.0,automatic,nc,nc,4
4,56.657400,-157.265100,0.50,2.90,ml,NaN,NaN,NaN,0.6300,ak,...,"80 km ENE of Chignik, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13318,42.057167,-120.038000,2.26,1.82,ml,7.0,164.00,0.25080,0.0800,uw,...,"22 km ENE of New Pine Creek, Oregon",earthquake,0.34,1.37,0.193501,2.0,reviewed,uw,uw,13319
13319,-12.226100,167.045400,235.32,4.50,mb,NaN,122.00,3.20500,0.9900,us,...,"190 km NNW of Sola, Vanuatu",earthquake,12.90,5.70,0.104000,27.0,reviewed,us,us,13320
13320,59.160800,-149.073100,30.10,1.90,ml,NaN,NaN,NaN,1.0400,ak,...,"103 km SSE of Lowell Point, Alaska",earthquake,NaN,4.00,NaN,NaN,automatic,ak,ak,13321
13321,43.155500,-122.949667,-0.52,0.92,md,4.0,304.00,0.18670,0.2500,uw,...,"20 km SE of Glide, Oregon",explosion,1.93,31.61,0.079716,3.0,reviewed,uw,uw,13322


In [24]:
table_df = pd.read_sql_table(
    "earthquake_types_predicted_data",
    con=engine
)
table_df

,Predicted Type,Actual Type,unique_id
0,1,1,1
1,1,1,2
2,1,1,3
3,1,1,4
4,1,1,5
...,...,...,...
3326,1,1,3327
3327,1,1,3328
3328,1,1,3329
3329,1,1,3330


In [35]:
connection.close()